In [1]:
import os
os.environ['COUNTLIMIT'] = '200000'
#default 250000

In [2]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [3]:
_ibpl = "Select (&CWV * [Location].[Location] * [EPM].[EPM] * [Time].[Week] * [Sales Domain].[Customer Group] * [Item].[L6].filter(#.Name in {\"175\", \"178\",\"184\",\"977\",\"958\",\"976\"}).relatedmembers([Planning Item]) ) on row, ({Measure.[Actual Shifted]}) on column;"

from o9_common_utils.O9DataLake import O9DataLake, ResourceType, DataSource
O9DataLake.register("df",DataSource.LS, ResourceType.IBPL, _ibpl)


In [4]:
import time
start = time.time()
df = O9DataLake.get("df")
end = time.time()
print(end - start)

2022-11-08 09:29:34,934 : nitishjha_6602 :IbplClient: WebApi Request Url = [https://mygenieazure.o9solutions.com/api/ibplquery/6602/ExecuteCompactJsonQuery]
2022-11-08 09:29:40,103 : nitishjha_6602 :IbplClient: WebApi Request Url = [https://mygenieazure.o9solutions.com/api/tenants/6602/batchjobs/fileDownloadBySelectQuery]
10.102627038955688


In [5]:
df.shape

(200000, 7)

In [6]:
df.head()

,Version.[Version Name],Location.[Location],EPM.[EPM],Time.[Week],Sales Domain.[Customer Group],Item.[Planning Item],Actual Shifted
0,CurrentWorkingView,9934,MO13GPHFSGHORW,2019-W19,182,951274643557803735,23323.590
1,CurrentWorkingView,9934,MO13GPHFSGHORW,2019-W20,182,951274643557803735,20448.135
2,CurrentWorkingView,9934,MO13GPHFSGHORW,2019-W21,182,951274643557803735,15975.135
3,CurrentWorkingView,9934,MO13GPHFSGHORW,2019-W22,182,951274643557803735,17892.000
4,CurrentWorkingView,9934,MO13GPHFSGHORW,2019-W23,182,951274643557803735,14377.590


In [7]:
df.dtypes

Version.[Version Name]            object
Location.[Location]                int64
EPM.[EPM]                         object
Time.[Week]                       object
Sales Domain.[Customer Group]     object
Item.[Planning Item]               int64
Actual Shifted                   float64
dtype: object

In [8]:
def convert_date(df):
    df["Time.Week"] = df["Time.[Week]"]
    df[['Time.[Week]','Week']] = df['Time.[Week]'].str.split('-',expand=True)
    df["Week"] = df['Week'].str.replace('W','')
    df["Time"] = df["Time.[Week]"] + df["Week"]
    df.drop(columns=["Week", "Time.[Week]"], inplace=True)
    df = df.astype({"Time": "int"})
    df.rename(columns={"Time.Week":"Time.[Week]"}, inplace=True)
    return df

df = convert_date(df)
df.head()

,Version.[Version Name],Location.[Location],EPM.[EPM],Sales Domain.[Customer Group],Item.[Planning Item],Actual Shifted,Time.[Week],Time
0,CurrentWorkingView,9934,MO13GPHFSGHORW,182,951274643557803735,23323.590,2019-W19,201919
1,CurrentWorkingView,9934,MO13GPHFSGHORW,182,951274643557803735,20448.135,2019-W20,201920
2,CurrentWorkingView,9934,MO13GPHFSGHORW,182,951274643557803735,15975.135,2019-W21,201921
3,CurrentWorkingView,9934,MO13GPHFSGHORW,182,951274643557803735,17892.000,2019-W22,201922
4,CurrentWorkingView,9934,MO13GPHFSGHORW,182,951274643557803735,14377.590,2019-W23,201923


In [9]:
X = df[["Time", "Location.[Location]", "EPM.[EPM]"]]
X = pd.get_dummies(data=X, drop_first=True)
Y = df["Actual Shifted"]

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .20, shuffle=False)
model = LinearRegression().fit(X_train, Y_train)
predict = model.predict(X_test)
r_sq = model.score(X_test, Y_test)
r_sq

-0.05862891968148176

In [11]:
df_out = df.filter(items=Y_test.index, axis=0)
df_out

,Version.[Version Name],Location.[Location],EPM.[EPM],Sales Domain.[Customer Group],Item.[Planning Item],Actual Shifted,Time.[Week],Time
160000,CurrentWorkingView,3542,MO13GPHNXGHQ,035,951274643557576835,4930.20,2022-W24,202224
160001,CurrentWorkingView,3542,MO13GPHNXGHQ,049,951274643557576835,1782.00,2022-W24,202224
160002,CurrentWorkingView,3542,MO13YJSWSUJJSB,028-YL58-33,951274643557576835,59.40,2022-W25,202225
160003,CurrentWorkingView,3542,MO13YVSWSUJJSB,C31-YL58-33,951274643557576835,1782.00,2022-W25,202225
160004,CurrentWorkingView,3542,MO13GPHNXGHQ,145,951274643557576835,1782.00,2022-W25,202225
...,...,...,...,...,...,...,...,...
199995,CurrentWorkingView,9909,MO13GRNNOHQNAB,OG4274562,951274643552488235,8252.40,2021-W13,202113
199996,CurrentWorkingView,9909,MO13GRNNOHQNAB,OG3274535,951274643552488235,2081.04,2021-W14,202114
199997,CurrentWorkingView,9909,MO13GRNNOHQNAB,OG4274562,951274643552488235,1722.24,2021-W14,202114
199998,CurrentWorkingView,9909,MO13GRNNOHQNAB,OG5274229,951274643552488235,3444.48,2021-W14,202114


In [12]:
df_out["PredictActualShifted"] = predict
df_out

,Version.[Version Name],Location.[Location],EPM.[EPM],Sales Domain.[Customer Group],Item.[Planning Item],Actual Shifted,Time.[Week],Time,PredictActualShifted
160000,CurrentWorkingView,3542,MO13GPHNXGHQ,035,951274643557576835,4930.20,2022-W24,202224,9349.948092
160001,CurrentWorkingView,3542,MO13GPHNXGHQ,049,951274643557576835,1782.00,2022-W24,202224,9349.948092
160002,CurrentWorkingView,3542,MO13YJSWSUJJSB,028-YL58-33,951274643557576835,59.40,2022-W25,202225,66207.611308
160003,CurrentWorkingView,3542,MO13YVSWSUJJSB,C31-YL58-33,951274643557576835,1782.00,2022-W25,202225,12050.999563
160004,CurrentWorkingView,3542,MO13GPHNXGHQ,145,951274643557576835,1782.00,2022-W25,202225,9356.581223
...,...,...,...,...,...,...,...,...,...
199995,CurrentWorkingView,9909,MO13GRNNOHQNAB,OG4274562,951274643552488235,8252.40,2021-W13,202113,3911.933773
199996,CurrentWorkingView,9909,MO13GRNNOHQNAB,OG3274535,951274643552488235,2081.04,2021-W14,202114,3918.566903
199997,CurrentWorkingView,9909,MO13GRNNOHQNAB,OG4274562,951274643552488235,1722.24,2021-W14,202114,3918.566903
199998,CurrentWorkingView,9909,MO13GRNNOHQNAB,OG5274229,951274643552488235,3444.48,2021-W14,202114,3918.566903


In [14]:
df_out.drop(columns=["Actual Shifted","Time"], inplace=True)

In [15]:
O9DataLake.put('df_out', df_out)